# Benchmarking

In [1]:
import sys
import time
import os
# sys.path.append('/Users/calum/Developer/pyzx-heuristics-master')
# import pyzx as zx_heuristics
sys.path.append('..')
import pyzx as zx
from benchmarking import benchmark

In [2]:
# b = benchmark(dirpath='benchmark_int')
b = benchmark()
b.load_circuits(os.path.join('..', 'circuits', 'benchmarking', 'Fast', 'before'), group_name='fast')
b.load_circuits(os.path.join('..', 'circuits', 'benchmarking', 'Fast', 'nrscm'), group_name='fast', simp_strategy='NRSCM')
b.show_attributes()

Circuit attributes:  ['Qubits', 'Gates', '2Q Count', 'T Count', 't_simp', 't_opt']
No loaded functions
Loaded routines:  ['NRSCM']
Loaded circuit groups:  ['fast']


,Original,NRSCM
fast,Y,Y


In [5]:
def int_c(c):
    t0 = time.time()
    g = c.to_graph()
    zx.simplify.to_gh(g)
    g2 = zx.simplify.teleport_reduce(g)
    zx.simplify.spider_simp(g2,quiet=True)
    zx.simplify.int_cliff_flow_simp(g2, x=[1,1,1,0,0,0,0,0,0],quiet=True)
    t1 = time.time()
    c2 = zx.extract.extract_simple(g2, up_to_perm=False).to_basic_gates()
    c3 = zx.optimize.basic_optimization(c2.copy(), do_swaps=False).to_basic_gates()
    c4 = zx.optimize.basic_optimization(c2.copy(), do_swaps=True).to_basic_gates()
    t2 = time.time()
    if c4.twoqubitcount() < c3.twoqubitcount(): return c4, t2-t1, t2-t0
    return c3, t2-t1, t2-t0

def int_c2(c):
    t0 = time.time()
    g = c.to_graph()
    zx.simplify.to_gh(g)
    g2 = zx.simplify.flow_reduce_int(g,x=[1,1,1,0,0,0,0,0,0], flow='causal',quiet=True)
    t1 = time.time()
    c2 = zx.extract.extract_simple(g2, up_to_perm=False).to_basic_gates()
    c3 = zx.optimize.basic_optimization(c2.copy(), do_swaps=False).to_basic_gates()
    c4 = zx.optimize.basic_optimization(c2.copy(), do_swaps=True).to_basic_gates()
    t2 = time.time()
    if c4.twoqubitcount() < c3.twoqubitcount(): return c4, t2-t1, t2-t0
    return c3, t2-t1, t2-t0

def int_g(c):
    t0 = time.time()
    g = c.to_graph()
    zx.simplify.to_gh(g)
    g2 = zx.simplify.teleport_reduce(g)
    zx.simplify.spider_simp(g2,quiet=True)
    zx.simplify.int_cliff_flow_simp(g2, x=[1,1,1,0,0,0,0,0,0],condition = lambda graph, match: True , quiet=True)
    t1 = time.time()
    c2 = zx.extract.extract_circuit(g2, up_to_perm=False).to_basic_gates()
    c3 = zx.optimize.basic_optimization(c2.copy(), do_swaps=False).to_basic_gates()
    c4 = zx.optimize.basic_optimization(c2.copy(), do_swaps=True).to_basic_gates()
    t2 = time.time()
    if c4.twoqubitcount() < c3.twoqubitcount(): return c4, t2-t1, t2-t0
    return c3, t2-t1, t2-t0

def int_g2(c):
    t0 = time.time()
    g = c.to_graph()
    zx.simplify.to_gh(g)
    g2 = zx.simplify.flow_reduce_int(g,x=[1,1,1,0,0,0,0,0,0], flow='g',quiet=True)
    t1 = time.time()
    c2 = zx.extract.extract_circuit(g2, up_to_perm=False).to_basic_gates()
    c3 = zx.optimize.basic_optimization(c2.copy(), do_swaps=False).to_basic_gates()
    c4 = zx.optimize.basic_optimization(c2.copy(), do_swaps=True).to_basic_gates()
    t2 = time.time()
    if c4.twoqubitcount() < c3.twoqubitcount(): return c4, t2-t1, t2-t0
    return c3, t2-t1, t2-t0

In [8]:
b.add_simplification_func(int_c, 'int_c', groups_to_run=['fast'],verify=True)
b.add_simplification_func(int_g, 'int_g', groups_to_run=['fast'],verify=True)
b.add_simplification_func(int_c2, 'int_c2', groups_to_run=['fast'],verify=True)
b.add_simplification_func(int_g2, 'int_g2', groups_to_run=['fast'],verify=True)

100%|██████████| 32/32 [00:00<00:00, 87495.26it/s]
Processing int_g on gf2^8_mult                                        :  72%|███████▏  | 23/32 [06:58<02:43, 18.21s/it]


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/pyzx/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/56/wdj2n1gs4cs1rm2hqmg1g0nm0000gn/T/ipykernel_10332/732865078.py", line 2, in <module>
    b.add_simplification_func(int_g, 'int_g', groups_to_run=['fast'],verify=True,rerun=True)
  File "/Users/calum/Developer/pyzx/optimisation/../benchmarking.py", line 125, in add_simplification_func
    if groups_to_run != None: self.run(funcs_to_run = [name], groups_to_run=groups_to_run, verify=verify, rerun=rerun)
  File "/Users/calum/Developer/pyzx/optimisation/../benchmarking.py", line 165, in run
    res = self.funcs[func_name](self.circuits[circ_name]['Original'][0])
  File "/var/folders/56/wdj2n1gs4cs1rm2hqmg1g0nm0000gn/T/ipykernel_10332/2991781833.py", line 22, in int_g
    zx.simplify.int_cliff_flow_simp(g2, x=[1,1,1,0,0,0,0,0,0],condition = 

In [9]:
b.show_attributes()

Circuit attributes:  ['Qubits', 'Gates', '2Q Count', 'T Count', 't_simp', 't_opt']
Loaded functions:  ['int_c', 'int_g']
Loaded routines:  ['NRSCM']
Loaded circuit groups:  ['fast']


,Original,NRSCM,int_c,int_g
fast,Y,Y,Y,Y


In [14]:
b.show_attributes()

Circuit attributes:  ['Qubits', 'Gates', '2Q Count', 'T Count', 't_simp', 't_opt']
Loaded functions:  ['int_cliff', 'int_cliff_tel', 'int_cliff_2Q', 'int_cliff2', 'int_cliff2_flow', 'nu_flow', 'Basic', 'full_reduce']
Loaded routines:  ['NRSCM']
Loaded circuit groups:  ['fast']


,Original,Basic,NRSCM,full_reduce,int_cliff,int_cliff2,int_cliff2_flow,int_cliff_2Q,int_cliff_tel,nu_flow
fast,Y,Y,Y,Y,Y,Y,Y,Y,Y,-


In [7]:
df = b.df(groups=['fast'],
     routines='all',
     funcs='all',
     atts=['Qubits', '2Q Count','T Count'])

In [5]:
b.save(dirpath='bm')